In [1]:
import pandas as pd
from collections import Counter
df = pd.read_csv('data/processed_claims.csv')
df.head()

,Claim,Food,Phenotype,Relationship-effect,Food Ontology Term,foodDB_wiki
0,ALA contributes to the maintenance of normal b...,Alpha-linolenic acid (ALA),Normal blood cholesterol,Maintenance of a function,https://foodb.ca/compounds/FDB012462,['http://en.wikipedia.org/wiki/Alpha-Linolenic...
1,Activated charcoal contributes to reducing exe...,Activated charcoal,Excessive flatulence,Enhancing a function,https://foodb.ca/compounds/FDB008898,['http://en.wikipedia.org/wiki/Activated_carbon']
2,Barley grain fibre contributes to an increase ...,Barley grain fibre,Increase in faecal bulk,Enhancing a function,https://foodb.ca/foods/FOOD00088,['http://en.wikipedia.org/wiki/Barley']
3,Beta-glucans contribute to the maintenance of ...,Beta-glucans,Normal blood cholesterol,Maintenance of a function,https://foodb.ca/compounds/FDB005762,['http://en.wikipedia.org/wiki/Glucan']
4,Betaine contributes to normal homocysteine met...,Betaine,Normal blood homocysteine,Maintenance of a function,https://foodb.ca/compounds/FDB009020,['http://en.wikipedia.org/wiki/Betaine']


# NER TASK

### ANOTATE CLAIMS USING OPENAI GPT-3.5 TURBO MODEL

The text present in the 'Claim' column is sent to the OpenAI GPT-3.5 Turbo model for annotation. The function extracts the entities, classifies them, and extracts an association between those entities. The entities to extract are of the types: "Food Entity", "Phenotype". The function returns the results as a YAML object with the following fields:

entities: the list of entities in the text, each entity is an object with the fields: label, type
association: a list with the most important association between entities in the text, an association is an object with the fields: "subject" for the subject entity, "predicate" for the relation (maintenance of function, enhancing a function, reducing a risk factor), "object" for the object entity
The function saves the annotated results in 'data/annotated_claims.csv' file.

In [2]:
import requests
import json

def annotateClaimsKG():
    url = 'https://api.collaboratory.semanticscience.org/openai-extract'
    headers = {'content-type': 'application/json'}
    params = {'prompt':'From the text below, extract the entities, classify them and extract an association between those entities, Entities to extract should be of one of those types: "Food Entity", "Phenotype". Return the results as a YAML object with the following fields: - entities: <the list of entities in the text, each entity is an object with the fields: label, type> - association: <a list with the most important association between entities in the text, an association is an object with the fields: "subject" for the subject entity, "predicate" for the relation (maintenance of function, enhancing a function, reducing a risk factor), "object" for the object entity>'}

    annot = []
    for i in range(0,len(df)):
        while True:
            data = {
                'text': df['Claim'][i]
            }
            response = requests.post(url, params=params, data=json.dumps(data), headers=headers)
            print(response)
            try:
                response = response.json()
                print(response)
                food_entities = [entity['label'] for entity in response['entities'] if entity['type'] == 'Food Entity']
                phenotype_entities = [entity['label'] for entity in response['entities'] if entity['type'] == 'Phenotype']
                health_relationships = [association['predicate'] for association in response['association']]
                row = [df['Claim'][i], food_entities, phenotype_entities, health_relationships]
                annot.append(row)
                break
            except:
                pass
    annotations = pd.DataFrame(annot, columns=['Claim', 'Food','Phenotype','Health relationship'])
    annotations.to_csv('data/annotated_claims.csv', index=False)
annotations = pd.read_csv('data/annotated_claims.csv')
annotations.head()

,Claim,Food,Phenotype,Health relationship
0,ALA contributes to the maintenance of normal b...,['ALA'],['normal blood cholesterol levels'],['contributes to the maintenance of']
1,Activated charcoal contributes to reducing exe...,"['Activated charcoal', 'eating']",['exessive flatulence'],['reducing a risk factor']
2,Barley grain fibre contributes to an increase ...,['Barley grain fibre'],['faecal bulk'],['contributes to an increase in']
3,Beta-glucans contribute to the maintenance of ...,['Beta-glucans'],['normal blood cholesterol levels'],['maintenance']
4,Betaine contributes to normal homocysteine met...,['Betaine'],['homocysteine metabolism'],['contributes to']


### EVALUATE ANNOTATIONS OF OPENAI GPT3.5-TURBO MODEL ON FOOD AND PHENOTYPES
Now we evaluate the annotations of the OpenAI GPT3.5-Turbo model on food terms using nervaluate. Full named-entity (i.e., not tag/token) evaluation metrics based on SemEval’13

In [3]:
import re
import ast
for i in range(len(df)):
    s = re.split(', | & | and | or ',df['Food'][i])
    s = [x for x in s if len(x) > 1]
    df['Food'][i] = s
    df['Phenotype'][i] = [df['Phenotype'][i]]
df.head()

,Claim,Food,Phenotype,Relationship-effect,Food Ontology Term,foodDB_wiki
0,ALA contributes to the maintenance of normal b...,[Alpha-linolenic acid (ALA)],[Normal blood cholesterol],Maintenance of a function,https://foodb.ca/compounds/FDB012462,['http://en.wikipedia.org/wiki/Alpha-Linolenic...
1,Activated charcoal contributes to reducing exe...,[Activated charcoal],[Excessive flatulence],Enhancing a function,https://foodb.ca/compounds/FDB008898,['http://en.wikipedia.org/wiki/Activated_carbon']
2,Barley grain fibre contributes to an increase ...,[Barley grain fibre],[Increase in faecal bulk],Enhancing a function,https://foodb.ca/foods/FOOD00088,['http://en.wikipedia.org/wiki/Barley']
3,Beta-glucans contribute to the maintenance of ...,[Beta-glucans],[Normal blood cholesterol],Maintenance of a function,https://foodb.ca/compounds/FDB005762,['http://en.wikipedia.org/wiki/Glucan']
4,Betaine contributes to normal homocysteine met...,[Betaine],[Normal blood homocysteine],Maintenance of a function,https://foodb.ca/compounds/FDB009020,['http://en.wikipedia.org/wiki/Betaine']


Convert the true and predicted labels to the indices that they match in the Claim text

In [4]:
from stringMatcher import longest_common_substring
# Convert "Food" column to indices
annotations_copy = annotations.copy()
df_copy = df[['Claim','Food','Phenotype']].copy()
annotations_copy['Food'] = annotations_copy.apply(lambda row: [longest_common_substring(row['Claim'],food) for food in ast.literal_eval(row['Food'])], axis=1)
df_copy['Food'] = df_copy.apply(lambda row: [longest_common_substring(row['Claim'],food) for food in row['Food']], axis=1)
# Convert "Phenotype" column to indices
annotations_copy['Phenotype'] = annotations_copy.apply(lambda row: [longest_common_substring(row['Claim'],phenotype) for phenotype in ast.literal_eval(row['Phenotype'])], axis=1)
df_copy['Phenotype'] = df_copy.apply(lambda row: [longest_common_substring(row['Claim'],phenotype) for phenotype in row['Phenotype']], axis=1)
# Display the updated DataFrame
indices_df = df_copy
indices_df['predicted_food'] = annotations_copy['Food']
indices_df['predicted_phenotype'] = annotations_copy['Phenotype']
indices_df.head()

,Claim,Food,Phenotype,predicted_food,predicted_phenotype
0,ALA contributes to the maintenance of normal b...,"[(0, 2)]","[(39, 61)]","[(0, 2)]","[(38, 68)]"
1,Activated charcoal contributes to reducing exe...,"[(0, 17)]","[(45, 61)]","[(0, 17), (69, 74)]","[(43, 61)]"
2,Barley grain fibre contributes to an increase ...,"[(0, 17)]","[(38, 59)]","[(0, 17)]","[(49, 59)]"
3,Beta-glucans contribute to the maintenance of ...,"[(0, 11)]","[(47, 69)]","[(0, 11)]","[(46, 76)]"
4,Betaine contributes to normal homocysteine met...,"[(0, 6)]","[(29, 41)]","[(0, 6)]","[(30, 52)]"


Add text length to the indices so the indices are not repeated for each row.

In [5]:
# Calculate the cumulative lengths of the Claim text
cumulative_lengths = indices_df['Claim'].apply(len).cumsum().shift(fill_value=0)
# Modify the spans in the columns based on the cumulative lengths
indices_df['Food'] = indices_df.apply(lambda row: [(start + cumulative_lengths[row.name], end + cumulative_lengths[row.name]) for (start, end) in row['Food']], axis=1)
indices_df['Phenotype'] = indices_df.apply(lambda row: [(start + cumulative_lengths[row.name], end + cumulative_lengths[row.name]) for (start, end) in row['Phenotype']], axis=1)
indices_df['predicted_food'] = indices_df.apply(lambda row: [(start + cumulative_lengths[row.name], end + cumulative_lengths[row.name]) for (start, end) in row['predicted_food']], axis=1)
indices_df['predicted_phenotype'] = indices_df.apply(lambda row: [(start + cumulative_lengths[row.name], end + cumulative_lengths[row.name]) for (start, end) in row['predicted_phenotype']], axis=1)
indices_df.head()

,Claim,Food,Phenotype,predicted_food,predicted_phenotype
0,ALA contributes to the maintenance of normal b...,"[(0, 2)]","[(39, 61)]","[(0, 2)]","[(38, 68)]"
1,Activated charcoal contributes to reducing exe...,"[(69, 86)]","[(114, 130)]","[(69, 86), (138, 143)]","[(112, 130)]"
2,Barley grain fibre contributes to an increase ...,"[(144, 161)]","[(182, 203)]","[(144, 161)]","[(193, 203)]"
3,Beta-glucans contribute to the maintenance of ...,"[(204, 215)]","[(251, 273)]","[(204, 215)]","[(250, 280)]"
4,Betaine contributes to normal homocysteine met...,"[(281, 287)]","[(310, 322)]","[(281, 287)]","[(311, 333)]"


Evaluate the annotations using nervaluate. Full named-entity (i.e., not tag/token) evaluation metrics based on SemEval’13. The strict metric is the strictest metric, which requires the predicted entity to match the true entity exactly. The type metric is the least strict metric, which requires the predicted entity to match the true entity partially.

True Positives (TP) ---> Correct
False Positives (FP) ---> Incorrect + Spurious
False Negatives (FN) ---> Incorrect + Missed
Nº of entities in the gold standard (manually annotated dataset) ---> Possible
Nº of entities predicted ---> Actual

In [6]:
food_true = [{'label': 'FOOD', 'start': span[0], 'end': span[1]} for spans in indices_df['Food'] for span in spans]
phenotype_true = [{'label': 'PHENOTYPE', 'start': span[0], 'end': span[1]} for spans in indices_df['Phenotype'] for span in spans]
food_predicted = [{'label': 'FOOD', 'start': span[0], 'end': span[1]} for spans in indices_df['predicted_food'] for span in spans]
phenotype_predicted = [{'label': 'PHENOTYPE', 'start': span[0], 'end': span[1]} for spans in indices_df['predicted_phenotype'] for span in spans]

true = [food_true, phenotype_true]
predicted = [food_predicted, phenotype_predicted]

from nervaluate import Evaluator
evaluator = Evaluator(true, predicted, tags=['FOOD', 'PHENOTYPE'])
# Returns overall metrics and metrics for each tag
results, results_per_tag = evaluator.evaluate()
food_ent_type_metrics = results_per_tag['FOOD']['ent_type']
food_strict_metrics = results_per_tag['FOOD']['strict']
phen_ent_type_metrics = results_per_tag['PHENOTYPE']['ent_type']
phen_strict_metrics = results_per_tag['PHENOTYPE']['strict']
print('FOOD','type',food_ent_type_metrics,'strict',food_strict_metrics)
print('PHENOTYPE','type',phen_ent_type_metrics,'strict',phen_strict_metrics)

FOOD type {'correct': 263, 'incorrect': 0, 'partial': 0, 'missed': 30, 'spurious': 35, 'possible': 293, 'actual': 298, 'precision': 0.8825503355704698, 'recall': 0.8976109215017065, 'f1': 0.8900169204737733} strict {'correct': 188, 'incorrect': 75, 'partial': 0, 'missed': 30, 'spurious': 35, 'possible': 293, 'actual': 298, 'precision': 0.6308724832214765, 'recall': 0.6416382252559727, 'f1': 0.6362098138747885}
PHENOTYPE type {'correct': 250, 'incorrect': 0, 'partial': 0, 'missed': 10, 'spurious': 135, 'possible': 260, 'actual': 385, 'precision': 0.6493506493506493, 'recall': 0.9615384615384616, 'f1': 0.7751937984496124} strict {'correct': 3, 'incorrect': 247, 'partial': 0, 'missed': 10, 'spurious': 135, 'possible': 260, 'actual': 385, 'precision': 0.007792207792207792, 'recall': 0.011538461538461539, 'f1': 0.009302325581395349}


# EL TASK

### EVALUATION OF ENTITY LINKING OF WIKIFIER FROM PREDICTED LABELS
First we query the Wikifier API to get the dbpedia iris and wikipedia urls of all the predicted food entities.

In [7]:
# get the dbpedia iris and wikipedia urls of all the predicted food entities
from wikifier import CallWikifier
def queryWikifier():
    data = []
    for i in range(len(annotations)):
        predicted_labels = ast.literal_eval(annotations['Food'][i])
        for plabel in predicted_labels:
            response = CallWikifier(plabel)
            wiki_annotations = response["annotations"]
            dbpedia_iris = []
            wiki_urls = []
            for wannotation in wiki_annotations:
                dbpedia_iri = wannotation["dbPediaIri"]
                wiki_url = wannotation["url"]
                wiki_urls.append(wiki_url)
                dbpedia_iris.append(dbpedia_iri)
            row = [annotations['Claim'][i], plabel, dbpedia_iris, wiki_urls]
            data.append(row)
    wikifier_annotations = pd.DataFrame(data, columns=['Claim', 'predicted_label','dbpedia_iris', 'wiki_urls'])
    wikifier_annotations.to_csv('data/wiki_iris_predicted.csv', index=False)

In [8]:
#Number of food terms with a Wikipedia URL
print('Number of unique food terms with fooDB ids',len(df['Food Ontology Term'].unique()))
print('Number of unique food terms with Wiki URLs',len(df['foodDB_wiki'].unique()))

Number of unique food terms with fooDB ids 60
Number of unique food terms with Wiki URLs 53


Then we compare the urls from Wikifier to the wikipedia urls of the food entities in the processed_claims dataset. If the urls are the same, then the entity linking is correct. We calculate the precision, recall, and F1-score.

Furthermore, we calculate the number of unique food terms correctly linked to a Wikipedia URL. We say that it is exact if the predicted Wikipedia URl is the same as the one in the dataset. We say it is partial if more than one Wikipedia pages were returned for that term and at least one of them is correct.

In [9]:
wiki_iris_predicted = pd.read_csv('data/wiki_iris_predicted.csv')
def evaluateEL(dataf):
    confusion_matrix = [[0,0],[0,0]]
    exact_unique_wikis = []
    nexact_unique_wikis = []
    for i in range(len(df)):
        true_labels = df['foodDB_wiki'][i]
        if pd.isna(true_labels):
            continue
        true_labels = ast.literal_eval(true_labels)
        predicted_labels = dataf[dataf['Claim'] == df['Claim'][i]]
        for predicted_label in predicted_labels['wiki_urls']:
            predicted_label = ast.literal_eval(predicted_label)
            for tlabel in true_labels:
                for plabel in predicted_label:
                    if plabel == tlabel:
                        if len(predicted_label) < 2:
                            exact_unique_wikis.append(tlabel)
                        else:
                            nexact_unique_wikis.append(tlabel)
                        confusion_matrix[0][0] += 1 # add to true positive
                        true_labels.remove(tlabel)
                        predicted_label.remove(plabel)
            confusion_matrix[0][1] += len(true_labels) # add to false negative (unmatched true labels)
            confusion_matrix[1][0] += len(predicted_label) # add to false positive (unmatched predicted labels)

    precision = confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[1][0])
    recall = confusion_matrix[0][0]/(confusion_matrix[0][0]+confusion_matrix[0][1])
    f1 = 2*(precision*recall)/(precision+recall)
    print(confusion_matrix)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1: ',f1)
    print('Unique terms exactly linked: ', len(Counter(exact_unique_wikis).keys()))
    print('Unique terms partially linked: ', len(Counter(nexact_unique_wikis).keys()))
evaluateEL(wiki_iris_predicted)

[[157, 82], [181, 0]]
Precision:  0.46449704142011833
Recall:  0.6569037656903766
F1:  0.5441941074523398
Unique terms exactly linked:  30
Unique terms partially linked:  9


### EVALUATION OF ENTITY LINKING OF WIKIFIER FROM CLAIM
In this approach the Wikifier API is called for each claim, and it annotates the relevant entities. Then the food terms are extracted by filtering the entities based on their dbepedia iris. If they are not of the type FOOD, CONDIMENT, BEVERAGE... they are removed.
Finally, the wikipedia urls of the remaining entities are compared to the true urls.

In [10]:
# Filter the predicted labels based on the dbpedia iris
from SPARQLWrapper import SPARQLWrapper, JSON
# get the dbpedia iris and wikipedia urls of claims
def queryWikifier():
    data = []
    for i in range(len(df)):
        claim = df['Claim'][i]
        response = CallWikifier(claim)
        wiki_annotations = response["annotations"]
        dbpedia_iris = []
        wiki_urls = []
        for wannotation in wiki_annotations:
            dbpedia_iri = wannotation["dbPediaIri"]
            wiki_url = wannotation["url"]
            wiki_urls.append(wiki_url)
            dbpedia_iris.append(dbpedia_iri)
        row = [annotations['Claim'][i], dbpedia_iris, wiki_urls]
        data.append(row)
    wikifier_annotations = pd.DataFrame(data, columns=['Claim','dbpedia_iris', 'wiki_urls'])
    wikifier_annotations.to_csv('data/wiki_iris.csv', index=False)

In [11]:
def queryDBpedia():
    wiki_iris = pd.read_csv('data/wiki_iris.csv')

    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setReturnFormat(JSON)

    for i in range(len(wiki_iris)):
        entities = ast.literal_eval(wiki_iris["dbpedia_iris"].iloc[i])
        wiki_urls = ast.literal_eval(wiki_iris["wiki_urls"].iloc[i])
        for entity in entities:
            query = f'''
                ASK {{
                {{<{entity}> rdf:type ?a.
                ?a rdfs:subClassOf* dbo:Food}}
                union
                {{<{entity}> rdf:type ?b.
                ?b rdfs:subClassOf* dbo:Condiment}}
                union
                {{<{entity}> rdf:type ?c.
                ?c rdfs:subClassOf* dbo:Beverage}}
                union
                {{<{entity}> rdf:type ?d.
                ?d rdfs:subClassOf* dbo:Plant}}
                union
                {{<{entity}> rdf:type ?e.
                ?e rdfs:subClassOf* dbo:Animal}}
                union
                {{<{entity}> rdf:type ?f.
                ?f rdfs:subClassOf* dbo:Fungus}}
                union
                {{<{entity}> rdf:type ?g.
                ?g rdfs:subClassOf* dbo:ChemicalSubstance}}
                }}
            '''
            sparql.setQuery(query)
            results = sparql.query().convert()
            isfood = results["boolean"]
            if  not isfood:
                index = entities.index(entity)
                entities.remove(entity)
                wiki_urls.remove(wiki_urls[index])
        print(wiki_urls)
        wiki_iris["dbpedia_iris"].iloc[i] = str(entities)
        wiki_iris["wiki_urls"].iloc[i] = str(wiki_urls)
    wiki_iris.to_csv('data/wiki_iris.csv', index=False)

In [12]:
wiki_iris = pd.read_csv('data/wiki_iris.csv')
evaluateEL(wiki_iris)

[[104, 123], [208, 0]]
Precision:  0.3333333333333333
Recall:  0.4581497797356828
F1:  0.38589981447124305
Unique terms exactly linked:  19
Unique terms partially linked:  15


# EXTRACTION OF RELATIONSHIP TYPE

### PROMPT 1

The predicted relations are the ones extracted at the beginning of the project in annotated_claims.csv. The relationships are mapped to the relations defined in the task (maintenance of a function, reducing a risk factor, enhancing of a function) with the exact wording.

In [13]:
def map_items(df, column_name):
    # Function to map items
    def mapper(item):
        if 'maintenance' in item:
            return 'Maintenance of a function'
        if 'contributes' in item:
            return 'Maintenance of a function'
        elif 'risk' in item:
            return 'Reducing a risk factor'
        elif 'enhancing' in item:
            return 'Enhancing a function'
        else:
            return item
    # Apply function to transform each cell in column
    df[column_name] = df[column_name].apply(lambda x: [mapper(i) for i in ast.literal_eval(x)])
    return df
health_rel = annotations.copy()
health_rel = map_items(health_rel, 'Health relationship')
health_rel.head()

,Claim,Food,Phenotype,Health relationship
0,ALA contributes to the maintenance of normal b...,['ALA'],['normal blood cholesterol levels'],[Maintenance of a function]
1,Activated charcoal contributes to reducing exe...,"['Activated charcoal', 'eating']",['exessive flatulence'],[Reducing a risk factor]
2,Barley grain fibre contributes to an increase ...,['Barley grain fibre'],['faecal bulk'],[Maintenance of a function]
3,Beta-glucans contribute to the maintenance of ...,['Beta-glucans'],['normal blood cholesterol levels'],[Maintenance of a function]
4,Betaine contributes to normal homocysteine met...,['Betaine'],['homocysteine metabolism'],[Maintenance of a function]


In [14]:
def evaluateRE(rel_df,column_name):
    A = [[0,0],[0,0]]
    B = [[0,0],[0,0]]
    C = [[0,0],[0,0]]
    for i in range(len(df)):
        true_rel = df['Relationship-effect'][i]
        pred_rels = rel_df[column_name][i]
        for pred_rel in pred_rels:
            matched = 0
            # if the true label is matched then is a true positive
            if true_rel == pred_rel:
                if true_rel == 'Maintenance of a function':
                    A[0][0] += 1
                    matched = 1
                elif true_rel == 'Reducing a risk factor':
                    B[0][0] += 1
                    matched = 1
                elif true_rel == 'Enhancing a function':
                    C[0][0] += 1
                    matched = 1
            # if the predicted label is wrong then it is a false positive
            else:
                if true_rel == 'Maintenance of a function':
                    A[1][0] += 1
                elif true_rel == 'Reducing a risk factor':
                    B[1][0] += 1
                elif true_rel == 'Enhancing a function':
                    C[1][0] += 1
        # if the true label is not matched by any predicted label then it is a false negative
        if matched == 0:
            if true_rel == 'Maintenance of a function':
                A[0][1] += 1
            elif true_rel == 'Reducing a risk factor':
                B[0][1] += 1
            elif true_rel == 'Enhancing a function':
                C[0][1] += 1

    precisionA = A[0][0]/(A[0][0]+A[1][0])
    recallA = A[0][0]/(A[0][0]+A[0][1])
    f1A = 2*(precisionA*recallA)/(precisionA+recallA)
    print('Maintenance of a function')
    print(A)
    print('Precision: ', precisionA)
    print('Recall: ', recallA)
    print('F1: ',f1A)

    precisionB = B[0][0]/(B[0][0]+B[1][0])
    recallB = B[0][0]/(B[0][0]+B[0][1])
    f1B = 2*(precisionB*recallB)/(precisionB+recallB)
    print('Reducing a risk factor')
    print(B)
    print('Precision: ', precisionB)
    print('Recall: ', recallB)
    print('F1: ',f1B)

    precisionC = C[0][0]/(C[0][0]+C[1][0])
    recallC = C[0][0]/(C[0][0]+C[0][1])
    f1C = 2*(precisionC*recallC)/(precisionC+recallC)
    print('Enhancing a function')
    print(C)
    print('Precision: ', precisionC)
    print('Recall: ', recallC)
    print('F1: ',f1C)

    overall_tp = A[0][0]+B[0][0]+C[0][0]
    overall_fn = A[0][1]+B[0][1]+C[0][1]
    overall_fp = A[1][0]+B[1][0]+C[1][0]
    overall_precision = overall_tp/(overall_tp+overall_fp)
    overall_recall = overall_tp/(overall_tp+overall_fn)
    overall_f1 = 2*(overall_precision*overall_recall)/(overall_precision+overall_recall)
    print('Overall')
    print([[overall_tp,overall_fn],[overall_fp,0]])
    print('Precision: ', overall_precision)
    print('Recall: ', overall_recall)
    print('F1: ',overall_f1)
evaluateRE(health_rel, 'Health relationship')

Maintenance of a function
[[138, 60], [69, 0]]
Precision:  0.6666666666666666
Recall:  0.696969696969697
F1:  0.6814814814814815
Reducing a risk factor
[[27, 13], [28, 0]]
Precision:  0.4909090909090909
Recall:  0.675
F1:  0.5684210526315789
Enhancing a function
[[15, 32], [47, 0]]
Precision:  0.24193548387096775
Recall:  0.3191489361702128
F1:  0.27522935779816515
Overall
[[180, 105], [144, 0]]
Precision:  0.5555555555555556
Recall:  0.631578947368421
F1:  0.5911330049261083


### PROMPT 2

Now we will use a different prompt. First we annotate the relations using the prompt:

In [15]:
import requests
import json

def annotateClaimsKGREL():
    url = 'https://api.collaboratory.semanticscience.org/openai-extract'
    headers = {'content-type': 'application/json'}
    params = {'prompt':'From the text below, classify the text as: (maintenance of function, enhancing a function, reducing a risk factor). Return the results as a YAML object with the following fields: - relation: the type of relation (maintenance of function, enhancing a function, reducing a risk factor)'}

    annot = []
    for i in range(0,len(df)):
        while True:
            data = {
                'text': df['Claim'][i]
            }
            response = requests.post(url, params=params, data=json.dumps(data), headers=headers)
            print(response)
            try:
                response = response.json()
                print(response['relation'])
                health_relationships = response
                row = [df['Claim'][i],health_relationships['relation']]
                annot.append(row)
                break
            except:
                pass
    annotations = pd.DataFrame(annot, columns=['Claim', 'Rel'])
    annotations.to_csv('data/relations.csv', index=False)
relations = pd.read_csv('data/relations.csv')
#relations['Rel'] = relations['Rel'].apply(lambda x: ast.literal_eval(x)[0]['predicate'])
relations.head()

,Claim,Rel
0,ALA contributes to the maintenance of normal b...,maintenance of function
1,Activated charcoal contributes to reducing exe...,reducing a risk factor
2,Barley grain fibre contributes to an increase ...,enhancing a function
3,Beta-glucans contribute to the maintenance of ...,maintenance of function
4,Betaine contributes to normal homocysteine met...,enhancing a function


Then we want to map all the relations to the relations defined in the task (maintenance of a function, reducing a risk factor, enhancing of a function) with the exact wording.

In [16]:
relations.loc[relations['Rel'].str.contains('maintenance', case=False), 'Rel'] = 'Maintenance of a function'
relations.loc[relations['Rel'].str.contains('risk', case=False), 'Rel'] = 'Reducing a risk factor'
relations.loc[relations['Rel'].str.contains('enhancing', case=False), 'Rel'] = 'Enhancing a function'
relations['Rel'] = relations['Rel'].apply(lambda x: [x])
relations.head()

,Claim,Rel
0,ALA contributes to the maintenance of normal b...,[Maintenance of a function]
1,Activated charcoal contributes to reducing exe...,[Reducing a risk factor]
2,Barley grain fibre contributes to an increase ...,[Enhancing a function]
3,Beta-glucans contribute to the maintenance of ...,[Maintenance of a function]
4,Betaine contributes to normal homocysteine met...,[Enhancing a function]


Now we evaluate the model

In [17]:
evaluateRE(relations, 'Rel')

Maintenance of a function
[[148, 36], [36, 0]]
Precision:  0.8043478260869565
Recall:  0.8043478260869565
F1:  0.8043478260869565
Reducing a risk factor
[[23, 10], [10, 0]]
Precision:  0.696969696969697
Recall:  0.696969696969697
F1:  0.696969696969697
Enhancing a function
[[22, 18], [18, 0]]
Precision:  0.55
Recall:  0.55
F1:  0.55
Overall
[[193, 64], [64, 0]]
Precision:  0.7509727626459144
Recall:  0.7509727626459144
F1:  0.7509727626459144
